**We’ll extract weather information about Pune, India from this [page](https://weather.com/en-IN/weather/tenday/l/INXX0634:1:IN). **

As you can see from the image, the page has information about the extended forecast for the next week, including time of day, temperature, and a brief description of the conditions.

**Exploring page structure with Chrome DevTools**

The first thing we’ll need to do is inspect the page using Chrome Devtools. If you’re using another browser, Firefox and Safari have equivalents. It’s recommended to use Chrome though.

You can start the developer tools in Chrome by clicking ***View -> Developer -> Developer Tools***. You should end up with a panel at the bottom of the browser like what you see below. Make sure the _Elements_ panel is highlighted:

We can then scroll up in the elements panel to find the “outermost” element that contains all of the text that corresponds to the extended forecasts. In this case, it’s a ***div*** tag with the id ***twc-scrollabe***:

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import date
from matplotlib import pyplot as plt

In [2]:
# getting current date

today = date.today() 

In [3]:
# page request for scrapping

url = "https://weather.com/en-IN/weather/tenday/l/INXX0634:1:IN"
page = requests.get(url) 

In [4]:
# parse the scrapped page using Python BeautifulSoup Parser
soup = BeautifulSoup(page.content, 'html.parser') 

# identifying the tag of the HTML page which has weather data
ten_day = soup.find_all('div', class_="twc-table-scroller") 

In [5]:
data = []
table = soup.find('table', attrs={'class':'twc-table'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')      
    cols = [element.text.strip() for element in cols]
    
    regex = re.compile(r'(\d.*)')
    day_date  = regex.split(cols[1].replace('\n',''))
    
    day = day_date[0]
    date = day_date[1] + ", " + str(today.year)
    high_temp = (cols[3][0:2]) if (cols[3][0:2] !='--') else (None)
    low_temp = cols[3][-3:].replace('°','')
    desc = cols[2]
    precip = cols[4]
    wind = cols[5]
    humidity = cols[6]
    
    # weather details of each day
    record = [day ,  date,  desc , high_temp , low_temp , precip , wind , humidity ]
    
    # store weather details of each day into a list
    data.append(record) 

### creating pandas dataframe from the above data

In [6]:
WeatherData = pd.DataFrame(data)
WeatherData.columns = ["DAY", "DATE", "DESCRIPTION",  "HIGH_TEMP","LOW_TEMP", "PRECIP", "WIND", "HUMIDITY"]
print(WeatherData)

        DAY          DATE    DESCRIPTION HIGH_TEMP LOW_TEMP PRECIP  \
0   Tonight  27 OCT, 2017  Partly Cloudy      None       20     0%   
1       Sat  28 OCT, 2017          Sunny        32       20     0%   
2       Sun  29 OCT, 2017   Mostly Sunny        32       19     0%   
3       Mon  30 OCT, 2017          Sunny        32       19     0%   
4       Tue  31 OCT, 2017          Sunny        32       18     0%   
5       Wed   1 NOV, 2017          Sunny        32       18     0%   
6       Thu   2 NOV, 2017          Sunny        31       18     0%   
7       Fri   3 NOV, 2017          Sunny        31       18     0%   
8       Sat   4 NOV, 2017          Sunny        31       18     0%   
9       Sun   5 NOV, 2017          Sunny        31       18     0%   
10      Mon   6 NOV, 2017          Sunny        31       20     0%   
11      Tue   7 NOV, 2017          Sunny        31       20     0%   
12      Wed   8 NOV, 2017          Sunny        31       19     0%   
13      Thu   9 NOV,

### Displaying data frame as HTML table

In [7]:
from IPython.display import display, HTML
HTML(WeatherData.to_html())

,DAY,DATE,DESCRIPTION,HIGH_TEMP,LOW_TEMP,PRECIP,WIND,HUMIDITY
0,Tonight,"27 OCT, 2017",Partly Cloudy,None,20,0%,SSE 1 km/h,85%
1,Sat,"28 OCT, 2017",Sunny,32,20,0%,NE 9 km/h,53%
2,Sun,"29 OCT, 2017",Mostly Sunny,32,19,0%,E 13 km/h,47%
3,Mon,"30 OCT, 2017",Sunny,32,19,0%,ENE 21 km/h,45%
4,Tue,"31 OCT, 2017",Sunny,32,18,0%,ENE 17 km/h,44%
5,Wed,"1 NOV, 2017",Sunny,32,18,0%,ENE 14 km/h,43%
6,Thu,"2 NOV, 2017",Sunny,31,18,0%,ENE 13 km/h,44%
7,Fri,"3 NOV, 2017",Sunny,31,18,0%,E 18 km/h,42%
8,Sat,"4 NOV, 2017",Sunny,31,18,0%,E 21 km/h,41%
9,Sun,"5 NOV, 2017",Sunny,31,18,0%,E 19 km/h,42%


### Saving Data frame as csv file

In [8]:
WeatherData.to_csv(("./data/"+ str(today)+".csv"), sep='\t', encoding='utf-8')